In [2]:
from sklearn.datasets import load_digits
import numpy as np
from sklearn.model_selection import train_test_split

В этом задании будет использоваться датасет digits из sklearn.datasets. Оставьте последние 25% объектов для контроля качества, разделив X и y на X_train, y_train и X_test, y_test.

Целью задания будет реализовать самый простой метрический классификатор — метод ближайшего соседа, а также сравнить качество работы реализованного вами 1NN с RandomForestClassifier из sklearn на 1000 деревьях.

Задание 1

Реализуйте самостоятельно метод одного ближайшего соседа с евклидовой метрикой для задачи классификации. Можно не извлекать корень из суммы квадратов отклонений, т.к. корень — монотонное преобразование и не влияет на результат работы алгоритма.

Никакой дополнительной работы с признаками в этом задании делать не нужно — мы еще успеем этим заняться в других курсах. Ваша реализация может быть устроена следующим образом: можно для каждого классифицируемого объекта составлять список пар (расстояние до точки из обучающей выборки, метка класса в этой точке), затем сортировать этот список (по умолчанию сортировка будет сначала по первому элементу пары, затем по второму), а затем брать первый элемент (с наименьшим расстоянием).

Сортировка массива длиной N требует порядка N log N сравнений (строже говоря, она работает за O(N log N)). Подумайте, как можно легко улучшить получившееся время работы. Кроме простого способа найти ближайший объект всего за N сравнений, можно попробовать придумать, как разбить пространство признаков на части и сделать структуру данных, которая позволит быстро искать соседей каждой точки. За выбор метода поиска ближайших соседей в KNeighborsClassifier из sklearn отвечает параметр algorithm — если у вас уже есть некоторый бэкграунд в алгоритмах и структурах данных, вам может быть интересно познакомиться со структурами данных ball tree и kd tree.

Доля ошибок, допускаемых 1NN на тестовой выборке, — ответ в задании 1.

Задание 2

Теперь обучите на обучающей выборке RandomForestClassifier(n_estimators=1000) из sklearn. Сделайте прогнозы на тестовой выборке и оцените долю ошибок классификации на ней. Эта доля — ответ в задании 2. Обратите внимание на то, как соотносится качество работы случайного леса с качеством работы, пожалуй, одного из самых простых методов — 1NN. Такое различие — особенность данного датасета, но нужно всегда помнить, что такая ситуация тоже может иметь место, и не забывать про простые методы.

In [44]:
digits = load_digits()
X = digits.data
y = digits.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, shuffle = False)

In [45]:
print(f'Data size: {digits.data.shape[0]}')
print(f'Train and test size: {X_train.shape[0] + X_test.shape[0]}')

Data size: 1797
Train and test size: 1797


In [16]:
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean

In [46]:
X_untransf = digits.data

In [47]:
sc = StandardScaler()
sc.fit(X_untransf)
X_transf = sc.transform(X_untransf)

In [48]:
X_transf.shape

(1797, 64)

Задание 1

In [55]:
class oneNN:
    def fit(self, X_train, y_train):
        self.x = X_train
        self.y = y_train
        #
    def __predict_one(self, x_row):
        columns = x_row.shape[0]
        if columns != self.x.shape[1]:
            raise Exception('Check the size of features in train and test')
        pairs = []
        rows = self.x.shape[0]
        for point in range(rows):
            current_p = self.x[point, :]
            current_distance = euclidean(current_p, x_row)**2
            ans = self.y[point]
            pairs.append((current_distance, ans))
        pairs = sorted(pairs, key=lambda x: x[0])
        return pairs[0][1]
    def predict(self, x_test):
        rows = x_test.shape[0]
        predictions = []
        for row in range(rows):
            predictions.append(self.__predict_one(x_test[row,:]))
        return predictions

In [50]:
X_train_sc = sc.transform(X_train)
X_test_sc = sc.transform(X_test)

In [58]:
est = oneNN()
est.fit(X_train, y_train)
predictions = est.predict(X_test)

In [39]:
from sklearn.metrics import accuracy_score

In [59]:
score = accuracy_score(y_test, predictions)
print(f'Score: {score}')

Score: 0.9622222222222222


In [53]:
def write_ans_1(score):
    with open("task1.txt", 'w') as task1:
        task1.write(str(1 - score))

In [54]:
write_ans_1(score)

In [60]:
from sklearn.neighbors import KNeighborsClassifier

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [61]:
model = KNeighborsClassifier(1,metric='euclidean')
model.fit(X_train_sc, y_train)
pred = model.predict(X_test_sc)

In [62]:
accuracy_score(y_test, pred)

0.9466666666666667

In [66]:
from sklearn.ensemble import RandomForestClassifier

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [67]:
mod = RandomForestClassifier(1000)
mod.fit(X_train_sc, y_train)
preds = mod.predict(X_test_sc)

In [68]:
accuracy_score(y_test, preds)

0.9355555555555556

То есть качество хуже kNN

In [69]:
def write_ans_2(score):
    with open("task2.txt", 'w') as task2:
        task2.write(str(1 - score))

In [70]:
write_ans_2(accuracy_score(y_test, preds))